In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/data_preprocessed.zip

Streaming output truncated to the last 5000 lines.
  inflating: data_preprocessed/val/1351.png  
  inflating: data_preprocessed/val/1352.png  
  inflating: data_preprocessed/val/13528.png  
  inflating: data_preprocessed/val/1353.png  
  inflating: data_preprocessed/val/13543.png  
  inflating: data_preprocessed/val/13545.png  
  inflating: data_preprocessed/val/1355.png  
  inflating: data_preprocessed/val/13554.png  
  inflating: data_preprocessed/val/13556.png  
  inflating: data_preprocessed/val/13580.png  
  inflating: data_preprocessed/val/13585.png  
  inflating: data_preprocessed/val/13596.png  
  inflating: data_preprocessed/val/13621.png  
  inflating: data_preprocessed/val/13634.png  
  inflating: data_preprocessed/val/13647.png  
  inflating: data_preprocessed/val/13652.png  
  inflating: data_preprocessed/val/13658.png  
  inflating: data_preprocessed/val/13667.png  
  inflating: data_preprocessed/val/13671.png  
  inflating: data_preprocessed/val/13677.png  
  inflating: 

In [4]:
import tensorflow as tf
from tensorflow.keras import applications, layers, models, callbacks
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Optional

# 1. Custom Metrics
class HourMAE(tf.keras.metrics.Metric):
    def __init__(self, name='hour_mae', **kwargs):
        super().__init__(name=name, **kwargs)
        self.mae = tf.keras.metrics.MeanAbsoluteError()

    def update_state(self, y_true, y_pred, sample_weight=None):
        true_h = y_true[..., 0] * 12
        pred_h = y_pred[..., 0] * 12
        self.mae.update_state(true_h, pred_h)

    def result(self):
        return self.mae.result()

    def reset_state(self):
        self.mae.reset_state()

class MinuteMAE(tf.keras.metrics.Metric):
    def __init__(self, name='minute_mae', **kwargs):
        super().__init__(name=name, **kwargs)
        self.mae = tf.keras.metrics.MeanAbsoluteError()

    def update_state(self, y_true, y_pred, sample_weight=None):
        true_m = y_true[..., 1] * 60
        pred_m = y_pred[..., 1] * 60
        self.mae.update_state(true_m, pred_m)

    def result(self):
        return self.mae.result()

    def reset_state(self):
        self.mae.reset_state()

class SecondMAE(tf.keras.metrics.Metric):
    def __init__(self, name='second_mae', **kwargs):
        super().__init__(name=name, **kwargs)
        self.mae = tf.keras.metrics.MeanAbsoluteError()

    def update_state(self, y_true, y_pred, sample_weight=None):
        true_s = y_true[..., 2] * 60
        pred_s = y_pred[..., 2] * 60
        self.mae.update_state(true_s, pred_s)

    def result(self):
        return self.mae.result()

    def reset_state(self):
        self.mae.reset_state()

# 2. Model Architecture
def create_time_model(img_size=(224, 224), base_model_name='resnet50'):
    if base_model_name == 'resnet50':
        base_model = applications.ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(*img_size, 3))
    else:
        base_model = applications.VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(*img_size, 3))

    base_model.trainable = False

    inputs = layers.Input(shape=(*img_size, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(3, activation='linear')(x)  # [hour, minute, second]

    return models.Model(inputs, outputs)

# 3. Data Generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size=32, img_size=(224, 224), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.n = len(df)
        self.indices = np.arange(self.n)

        # Clean paths and normalize labels
        self.image_paths = df['new_img_dir'].str.replace('artifacts/', '/content/artifacts/').str.replace('\\', '/')
        self.labels = df[['hour', 'minute', 'second']].values / [12, 60, 60]

        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(self.n / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]

        X = np.empty((len(batch_indices), *self.img_size, 3), dtype=np.float32)
        y = np.empty((len(batch_indices), 3), dtype=np.float32)

        for i, idx in enumerate(batch_indices):
            X[i] = self._load_image(self.image_paths[idx])
            y[i] = self.labels[idx]

        return X, y

    def _load_image(self, path: str) -> np.ndarray:
        try:
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)
            return img.astype(np.float32) / 255.0
        except:
            return np.zeros((*self.img_size, 3), dtype=np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# 4. Training Components
def cyclic_time_loss(y_true, y_pred):
    true_time = y_true * [2*np.pi/12, 2*np.pi/60, 2*np.pi/60]
    pred_time = y_pred * [2*np.pi/12, 2*np.pi/60, 2*np.pi/60]
    return tf.reduce_mean(tf.abs(tf.sin((true_time - pred_time)/2)))

def train_model(train_gen, val_gen, img_size=(224, 224)):
    model = create_time_model(img_size)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=cyclic_time_loss,
        metrics=[HourMAE(), MinuteMAE(), SecondMAE()]
    )

    callbacks_lst = [
        tf.keras.callbacks.EarlyStopping(
            patience=15,
            monitor='val_hour_mae',
            mode='min',
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_clock_model.h5',
            monitor='val_hour_mae',
            save_best_only=True,
            mode='min',
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_hour_mae',
            factor=0.2,
            patience=5,
            mode='min',
            min_lr=1e-6
        )
    ]

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=100,
        callbacks=callbacks_lst
    )

    return model, history

# 5. Model Saving/Loading
def save_model(model, path='clock_model.h5'):
    model.save(path)

def load_model(path='clock_model.h5'):
    return models.load_model(
        path,
        custom_objects={
            'HourMAE': HourMAE,
            'MinuteMAE': MinuteMAE,
            'SecondMAE': SecondMAE,
            'cyclic_time_loss': cyclic_time_loss
        }
    )

import matplotlib.pyplot as plt
import os

def plot_training(history, save_dir='training_plots', save_name='training_metrics.png',
                 save_format='png', dpi=300, show=True):
    """
    Plot and save training metrics visualization
    """
    # Create directory if needed
    os.makedirs(save_dir, exist_ok=True)

    plt.figure(figsize=(15, 5))

    # Loss plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Loss Curve')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    # Hour MAE plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['hour_mae'], label='Train')
    plt.plot(history.history['val_hour_mae'], label='Validation')
    plt.title('Hour MAE')
    plt.ylabel('Hours MAE')
    plt.xlabel('Epoch')
    plt.legend()

    # Minute & Second MAE plot
    plt.subplot(1, 3, 3)
    plt.plot(history.history['minute_mae'], label='Min Train')
    plt.plot(history.history['val_minute_mae'], label='Min Val')
    plt.plot(history.history['second_mae'], label='Sec Train')
    plt.plot(history.history['val_second_mae'], label='Sec Val')
    plt.title('Minute & Second MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend()

    plt.tight_layout()

    # Save the full figure
    full_path = os.path.join(save_dir, save_name)
    plt.savefig(full_path, format=save_format, dpi=dpi, bbox_inches='tight')

    # Save individual components
    components = {
        'loss': (0, 'loss_curve'),
        'hour_mae': (1, 'hour_mae'),
        'minute_second_mae': (2, 'minute_second_mae')
    }

    for key, (idx, name) in components.items():
        fig = plt.figure(figsize=(5, 4))
        ax = fig.add_subplot(111)
        ax.set_position([0.15, 0.15, 0.75, 0.75])  # Adjust margins

        if key == 'loss':
            ax.plot(history.history['loss'], label='Train')
            ax.plot(history.history['val_loss'], label='Validation')
            ax.set_title('Loss Curve')
            ax.set_ylabel('Loss')
        elif key == 'hour_mae':
            ax.plot(history.history['hour_mae'], label='Train')
            ax.plot(history.history['val_hour_mae'], label='Validation')
            ax.set_title('Hour MAE')
            ax.set_ylabel('Hours MAE')
        else:
            ax.plot(history.history['minute_mae'], label='Min Train')
            ax.plot(history.history['val_minute_mae'], label='Min Val')
            ax.plot(history.history['second_mae'], label='Sec Train')
            ax.plot(history.history['val_second_mae'], label='Sec Val')
            ax.set_title('Minute & Second MAE')
            ax.set_ylabel('MAE')

        ax.set_xlabel('Epoch')
        ax.legend()
        plt.savefig(os.path.join(save_dir, f'{name}.{save_format}'),
                    format=save_format, dpi=dpi)
        plt.close(fig)

    if show:
        plt.show()
    else:
        plt.close()

    print(f"Saved training plots to: {save_dir}")

# 7. Inference
def predict_time(model, image_path, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    hour = (pred[0] * 12) % 12
    minute = (pred[1] * 60) % 60
    second = (pred[2] * 60) % 60

    return hour, minute, second

# Usage Example
if __name__ == "__main__":
    # Load and prepare data
    train_df = pd.read_csv("/content/artifacts/data_preprocessed/train.csv")
    val_df = pd.read_csv("/content/artifacts/data_preprocessed/val.csv")

    train_gen = CustomDataGenerator(train_df)
    val_gen = CustomDataGenerator(val_df, shuffle=False)

    # Train
    model, history = train_model(train_gen, val_gen)
    plot_training(
          history,
          save_dir='experiment_1_plots',
          save_name='training_metrics.svg',
          save_format='svg',
          dpi=300,
          show=False
      )

    # Save
    save_model(model)

    # Predict
    hour, minute, second = predict_time(model, "/content/artifacts/data_preprocessed/test/10009.png")
    print(f"Predicted Time: {int(hour):02}:{int(minute):02}:{int(second):02}")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - hour_mae: 3.1034 - loss: 0.0317 - minute_mae: 15.8322 - second_mae: 15.8828
Epoch 1: val_hour_mae improved from inf to 2.63511, saving model to best_clock_model.h5


976/976 ━━━━━━━━━━━━━━━━━━━━ 116s 108ms/step - hour_mae: 3.1029 - loss: 0.0317 - minute_mae: 15.8301 - second_mae: 15.8817 - val_hour_mae: 2.6351 - val_loss: 0.0271 - val_minute_mae: 13.1207 - val_second_mae: 14.3242 - learning_rate: 0.0010
Epoch 2/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 2.3269 - loss: 0.0248 - minute_mae: 13.0274 - second_mae: 14.1594
Epoch 2: val_hour_mae improved from 2.63511 to 2.01068, saving model to best_clock_model.h5


976/976 ━━━━━━━━━━━━━━━━━━━━ 127s 96ms/step - hour_mae: 2.3268 - loss: 0.0248 - minute_mae: 13.0271 - second_mae: 14.1593 - val_hour_mae: 2.0107 - val_loss: 0.0220 - val_minute_mae: 11.8728 - val_second_mae: 13.6324 - learning_rate: 0.0010
Epoch 3/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 2.1064 - loss: 0.0230 - minute_mae: 12.5592 - second_mae: 13.9561
Epoch 3: val_hour_mae did not improve from 2.01068
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 96ms/step - hour_mae: 2.1063 - loss: 0.0230 - minute_mae: 12.5591 - second_mae: 13.9561 - val_hour_mae: 2.0516 - val_loss: 0.0220 - val_minute_mae: 11.3860 - val_second_mae: 13.1452 - learning_rate: 0.0010
Epoch 4/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 2.0179 - loss: 0.0223 - minute_mae: 12.3700 - second_mae: 13.7489
Epoch 4: val_hour_mae improved from 2.01068 to 1.85839, saving model to best_clock_model.h5


...

976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 96ms/step - hour_mae: 1.5232 - loss: 0.0183 - minute_mae: 12.0008 - second_mae: 12.8851 - val_hour_mae: 1.0475 - val_loss: 0.0142 - val_minute_mae: 10.8176 - val_second_mae: 11.8620 - learning_rate: 0.0010
Epoch 22/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.5003 - loss: 0.0181 - minute_mae: 12.1152 - second_mae: 12.7587
Epoch 22: val_hour_mae improved from 1.04746 to 1.03959, saving model to best_clock_model.h5


976/976 ━━━━━━━━━━━━━━━━━━━━ 142s 96ms/step - hour_mae: 1.5003 - loss: 0.0181 - minute_mae: 12.1152 - second_mae: 12.7587 - val_hour_mae: 1.0396 - val_loss: 0.0143 - val_minute_mae: 10.8507 - val_second_mae: 12.4074 - learning_rate: 0.0010
Epoch 23/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.4939 - loss: 0.0180 - minute_mae: 11.8435 - second_mae: 12.7700
Epoch 23: val_hour_mae did not improve from 1.03959
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 96ms/step - hour_mae: 1.4939 - loss: 0.0180 - minute_mae: 11.8435 - second_mae: 12.7699 - val_hour_mae: 1.1217 - val_loss: 0.0148 - val_minute_mae: 10.6326 - val_second_mae: 12.3794 - learning_rate: 0.0010
Epoch 24/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.4865 - loss: 0.0179 - minute_mae: 11.7866 - second_mae: 12.5537
Epoch 24: val_hour_mae did not improve from 1.03959
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 96ms/step - hour_mae: 1.4866 - loss: 0.0179 - minute_mae: 11.7867 - second_mae: 12.5538 - val_hour_mae: 1.0820 - val_l

976/976 ━━━━━━━━━━━━━━━━━━━━ 143s 97ms/step - hour_mae: 1.4840 - loss: 0.0179 - minute_mae: 11.9169 - second_mae: 12.5104 - val_hour_mae: 1.0334 - val_loss: 0.0138 - val_minute_mae: 10.3355 - val_second_mae: 11.4077 - learning_rate: 0.0010
Epoch 26/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.4837 - loss: 0.0179 - minute_mae: 11.8510 - second_mae: 12.6419
Epoch 26: val_hour_mae did not improve from 1.03338
976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 96ms/step - hour_mae: 1.4837 - loss: 0.0179 - minute_mae: 11.8510 - second_mae: 12.6417 - val_hour_mae: 1.3857 - val_loss: 0.0163 - val_minute_mae: 10.4413 - val_second_mae: 11.2032 - learning_rate: 0.0010
Epoch 27/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - hour_mae: 1.5081 - loss: 0.0180 - minute_mae: 11.7846 - second_mae: 12.4618
Epoch 27: val_hour_mae improved from 1.03338 to 0.99080, saving model to best_clock_model.h5


976/976 ━━━━━━━━━━━━━━━━━━━━ 150s 104ms/step - hour_mae: 1.5081 - loss: 0.0180 - minute_mae: 11.7845 - second_mae: 12.4619 - val_hour_mae: 0.9908 - val_loss: 0.0135 - val_minute_mae: 10.6801 - val_second_mae: 11.0767 - learning_rate: 0.0010
Epoch 28/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.4684 - loss: 0.0177 - minute_mae: 11.7782 - second_mae: 12.4126
Epoch 28: val_hour_mae did not improve from 0.99080
976/976 ━━━━━━━━━━━━━━━━━━━━ 94s 96ms/step - hour_mae: 1.4684 - loss: 0.0177 - minute_mae: 11.7781 - second_mae: 12.4125 - val_hour_mae: 1.0808 - val_loss: 0.0141 - val_minute_mae: 10.5112 - val_second_mae: 11.0124 - learning_rate: 0.0010
Epoch 29/100
976/976 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - hour_mae: 1.4432 - loss: 0.0175 - minute_mae: 11.8029 - second_mae: 12.2792
Epoch 29: val_hour_mae did not improve from 0.99080
976/976 ━━━━━━━━━━━━━━━━━━━━ 93s 96ms/step - hour_mae: 1.4432 - loss: 0.0175 - minute_mae: 11.8028 - second_mae: 12.2792 - val_hour_mae: 1.0400 - val_

Saved training plots to: experiment_1_plots
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Time: 04:28:39


In [7]:
%pwd

'd:\\Advanced Project\\time-estimation\\notebooks'

In [8]:
import os
os.chdir("../")

In [9]:
%pwd

'd:\\Advanced Project\\time-estimation'

In [27]:
from dataclasses import dataclass
from pathlib import Path

In [28]:
@dataclass(frozen=True)
class ModelTrainingEntity:
    root_dir: Path
    train_csv: Path
    val_csv: Path
    model_performance: Path
    final_model: Path
    checkpoint_model: Path
    img_height: int
    img_width: int
    learning_rate: float
    epochs: int
    batch_size: int

In [29]:
from timeEstimator.constant import *
from timeEstimator.Utils.common import read_yaml, create_directory

In [30]:
class ConfigurationManager:
    def __init__(self, params=PARAMS_FILE_PATH, config=CONFIG_FILE_PATH):
        self.params = read_yaml(params)
        self.config = read_yaml(config)

        create_directory([self.config.root_dir])

    def model_training_config(self):
        config = self.config.model_training
        params = self.params
        create_directory([config.root_dir])
        entity = ModelTrainingEntity(
            root_dir = config.root_dir,
            train_csv = config.train_csv,
            val_csv = config.val_csv,
            model_performance = config.model_performance,
            final_model = config.final_model,
            checkpoint_model = config.checkpoint_model,
            img_height = params.img_height,
            img_width = params.img_width,
            learning_rate = params.learning_rate,
            epochs = params.epochs,
            batch_size = params.batch_size
        )

        return entity

In [38]:
from timeEstimator.model.custom_resnet import create_time_model
from timeEstimator.model.metrics import HourMAE, MinuteMAE, SecondMAE
from timeEstimator.model.loss import cyclic_time_loss
from timeEstimator.Utils.common import CustomDataGenerator
import pandas as pd
import tensorflow as tf

In [39]:
class ModelTraining:
    def __init__(self, config: ModelTrainingEntity):
        self.config = config

    def train_model(self, train_gen, val_gen, img_size):
        
        model = create_time_model(img_size)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(1e-3),
            loss=cyclic_time_loss,
            metrics=[HourMAE(), MinuteMAE(), SecondMAE()]
        )

        callbacks_lst = [
            tf.keras.callbacks.EarlyStopping(
                patience=15,
                monitor='val_hour_mae',
                mode='min',
                restore_best_weights=True
            ),
            tf.keras.callbacks.ModelCheckpoint(
                self.config.checkpoint_model,
                monitor='val_hour_mae',
                save_best_only=True,
                mode='min',
                verbose=1
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_hour_mae',
                factor=0.2,
                patience=5,
                mode='min',
                min_lr=1e-6
            )
        ]

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=self.config.epochs,
            callbacks=callbacks_lst
        )

        return model, history

    def save_model(self, model, path):
        path = self.config.final_model
        model.save(path)

    def model_train(self):
        train_df = pd.read_csv("artifacts/data_preprocessed/train.csv")
        val_df = pd.read_csv("artifacts/data_preprocessed/val.csv")

        train_gen = CustomDataGenerator(df=train_df,batch_size=self.config.batch_size)
        val_gen = CustomDataGenerator(df=val_df, batch_size=self.config.batch_size,shuffle=False)

        # Train
        img_size = (self.config.img_height, self.config.img_width)
        model, history = self.train_model(train_gen, val_gen, img_size)
        self.plot_training(
            history,
            save_dir=self.config.model_performance,
            save_name='training_metrics.svg',
            save_format='svg',
            dpi=300,
            show=False
        )

        # Save
        self.save_model(model)


In [40]:
try:
    config = ConfigurationManager()
    training_config = config.model_training_config()
    model_training = ModelTraining(training_config)
    model_training.model_train()
except Exception as e:
    raise e

[2025-02-26 12:34:19,078]: INFO: common : Read YAML File: params.yaml
[2025-02-26 12:34:19,081]: INFO: common : Read YAML File: config\config.yaml
[2025-02-26 12:34:19,082]: INFO: common : Directory has been Created: artifacts
[2025-02-26 12:34:19,084]: INFO: common : Directory has been Created: model


d:\Advanced Project\time-estimation\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
  5/976 ━━━━━━━━━━━━━━━━━━━━ 23:25 1s/step - hour_mae: 14.9618 - loss: 0.1357 - minute_mae: 66.4570 - second_mae: 40.8796

KeyboardInterrupt: 